In [ ]:
# Importing the Libraries
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests

import time

from IPython.display import Audio
sound_file = './beep.wav'

import sys
from termcolor import colored, cprint

In [ ]:
Audio(sound_file, autoplay=True)

In [ ]:
def page_soup(page_number):
    url = "https://thistudy.com/index.php?board=162."+str(page_number*20)
    #Making a GET requests to fetch the raw HTML content
    # headers = {
    #     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    # }
    # headers={
    #   'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #   'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7',
    #   'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    # }

    r = requests.get(url)
    c = r.content
    #Parsing the HTML content
    soup = BeautifulSoup(c,"html.parser")
    
    while soup.find("title").text =='Verify, Please!':
        alert = "!"*20,"page_number ",str(page_number),"\t Captcha Error","!"*20
        alert = colored(alert,"red",attrs=["reverse","blink"])
        print(alert)
        Audio(sound_file, autoplay=True)
        time.sleep(30)
        #Again Request That Page
        r = requests.get(url)
        c = r.content
        #Parsing the HTML content
        soup = BeautifulSoup(c,"html.parser")
    
        
    text = "Page Number\t"+str(page_number)+"..."
    
    text_color = colored(text, 'green', attrs=['reverse', 'blink'])
    print(text_color)
    
    
    return soup
    
def table_soup(soup):
    table = soup.find("table")
    return table

def question(table):
#     question_link = lambda y:y["href"]
#     question_text = lambda y:y.text
    question_list = table.find_all("a")
    
    ques_text = [question_list[i].text for i in range(4,len(question_list),2)]
    ques_link = [question_list[i]["href"] for i in range(4,len(question_list),2)]
    
    return ques_text, ques_link

def views(table):
    views = table.find_all("li")
    view_list = [views[i].text.split()[0] for i in range(1,len(views),2)]
    
    return view_list
    
def scrape_list(page_number):
    time.sleep(1)
    soup = page_soup(page_number)
    table = table_soup(soup)
    ques_text,ques_link= question(table)
    view = views(table)
    return ques_text,ques_link,view


    
    
    

In [ ]:
# Question Link, text and views Extraction
df = {"question":[],"links":[],"views":[]}

for page_number in range(231):
    ques_text,ques_link,view = scrape_list(page_number)
    
    for i in range(len(ques_text)):
        df["question"].append(ques_text[i])
        df["links"].append(ques_link[i])
        df["views"].append(view[i])
    
        
df = pd.DataFrame.from_dict(df)
df
    

In [ ]:
df.to_csv("scrape.csv")

In [ ]:
def answer_soup(url):
    url = url
    #Making a GET requests to fetch the raw HTML content
    # headers = {
    #     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    # }
    # headers={
    #   'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #   'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7',
    #   'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    # }
    
    try :
        r = requests.get(url)
        c = r.content
        #Parsing the HTML content
        soup = BeautifulSoup(c,"html.parser")
        return soup
    except:
        print("|||||||||||||||||||Captcha Error|||||||||||||||||||||")
        pass
    

In [ ]:
df = pd.read_csv("scrape.csv",index_col =0)
df.head()

In [ ]:
def answer_soup(url,ques_number):
    url = url
    #Making a GET requests to fetch the raw HTML content
    # headers = {
    #     'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    # }
    # headers={
    #   'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    #   'Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,ja;q=0.7',
    #   'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36'
    # }
    r = requests.get(url)
    c = r.content
    #Parsing the HTML content
    soup = BeautifulSoup(c,"html.parser")
    
    while soup.find("title").text =='Verify, Please!':
        alert = "!"*20,"ques_number ",str(ques_number),"\t Captcha Error","!"*20
        alert = colored(alert,"red",attrs=["reverse","blink"])
        print(alert)
        Audio(sound_file, autoplay=True)
        time.sleep(30)
        #Again Request That Page
        r = requests.get(url)
        c = r.content
        #Parsing the HTML content
        soup = BeautifulSoup(c,"html.parser")
    
    text = "Question Number\t"+str(ques_number) +"..."
    
    text_color = colored(text, 'green', attrs=['reverse', 'blink'])
    print(text_color)
    
    
    return soup

def ques_answer(url,ques_number):
    soup = answer_soup(url,ques_number)
    soup = soup.find("article")
    div = soup.find_all("div")
    answer = soup.find("div",{"itemprop":"suggestedAnswer acceptedAnswer"})
    answer = answer.find("div",{"itemprop":"text"})
    ans = answer.text.strip()
    
    return ans
    
    

In [ ]:
df.info()

In [ ]:
answer_list = []
for i in range(4607):
    ques_number = i
    url = df["links"].iloc[i]
    answer_list.append(ques_answer(url,ques_number))

In [ ]:
answer.text.strip()

In [ ]:
answer_df = pd.DataFrame(answer_list)
answer_df.to_csv("answer_list")
answer_list

In [ ]:
df["answer_list"] = x4
df.info()